## Migros

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import asyncio
# from playwright.sync_api import Page, expect, async_playwright
from playwright.async_api import async_playwright


In [8]:
# !playwright install

In [19]:
# playwright = await async_playwright().start()
# browser = await playwright.chromium.launch(headless = True)
# page = await browser.new_page()


In [2]:
# query = 'data scientist'
await page.goto("https://filialen.migros.ch/de/center:46.8182,8.2275/zoom:8/")

<Response url='https://filialen.migros.ch/de/center:46.8182,8.2275/zoom:8' request=<Request url='https://filialen.migros.ch/de/center:46.8182,8.2275/zoom:8' method='GET'>>

In [13]:
# await page.screenshot(path="screenshot.png")

In [3]:
elements = await page.query_selector_all("*")

In [4]:
# elements

In [4]:
buttons = await page.query_selector_all("button")

In [18]:
import pandas as pd
from playwright.async_api import async_playwright
import nest_asyncio
nest_asyncio.apply()

async def scrape_all_migros_stores():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        await page.goto("https://filialen.migros.ch/de/")

        # Open the list view
        toggle_btn = page.locator("[data-test-id='storefinder_toggle-list-button']")
        await toggle_btn.wait_for()
        await toggle_btn.click(force=True)
        await toggle_btn.click(force=True)

        store_list_locator = page.locator("[data-test-id='storefinder_stores-list']")
        li_locator = store_list_locator.locator("li")

        print("🔃 Scrolling and scraping dynamically loaded list...")

        seen = set()
        store_data = []

        prev_count = -1
        stable_rounds = 0
        max_stable_rounds = 4

        while stable_rounds < max_stable_rounds:
            lis = await li_locator.all()
            new_count = 0

            for li in lis:
                name_el = li.locator("span[data-test-id*='stores-list_item-name']")
                address_el = li.locator("p[data-test-id*='stores-list_item-address']")

                if await name_el.count() == 0 or await address_el.count() == 0:
                    continue

                name = await name_el.inner_text()
                address = await address_el.inner_text()

                if name in seen:
                    continue

                seen.add(name)
                new_count += 1

                # Parse ZIP and Town
                zip_code, town = "", ""
                if "," in address:
                    parts = address.split(",")
                    if len(parts) == 2:
                        zip_town = parts[1].strip().split(" ", 1)
                        if len(zip_town) == 2:
                            zip_code, town = zip_town

                store_data.append({
                    "Store Name": name,
                    "Full Address": address,
                    "ZIP Code": zip_code,
                    "Town": town
                })

            print(f"🔁 Scroll {stable_rounds + 1}: Collected total {len(seen)} (new this round: {new_count})")

            if new_count == 0:
                stable_rounds += 1
            else:
                stable_rounds = 0

            await li_locator.last.hover()
            await page.mouse.wheel(0, 800)
            await page.wait_for_timeout(1000)

        await browser.close()

        df = pd.DataFrame(store_data)
        return df

# Run and display results
df_stores = await scrape_all_migros_stores()
import IPython.display as disp
disp.display(df_stores.head(10))
print(f"\n✅ Total unique stores scraped: {len(df_stores)}")


🔃 Scrolling and scraping dynamically loaded list...
🔁 Scroll 1: Collected total 14 (new this round: 14)
🔁 Scroll 1: Collected total 32 (new this round: 18)
🔁 Scroll 1: Collected total 48 (new this round: 16)
🔁 Scroll 1: Collected total 65 (new this round: 17)
🔁 Scroll 1: Collected total 82 (new this round: 17)
🔁 Scroll 1: Collected total 100 (new this round: 18)
🔁 Scroll 1: Collected total 117 (new this round: 17)
🔁 Scroll 1: Collected total 133 (new this round: 16)
🔁 Scroll 1: Collected total 148 (new this round: 15)
🔁 Scroll 1: Collected total 162 (new this round: 14)
🔁 Scroll 1: Collected total 180 (new this round: 18)
🔁 Scroll 1: Collected total 197 (new this round: 17)
🔁 Scroll 1: Collected total 214 (new this round: 17)
🔁 Scroll 1: Collected total 232 (new this round: 18)
🔁 Scroll 1: Collected total 249 (new this round: 17)
🔁 Scroll 1: Collected total 265 (new this round: 16)
🔁 Scroll 1: Collected total 282 (new this round: 17)
🔁 Scroll 1: Collected total 298 (new this round: 16)

,Store Name,Full Address,ZIP Code,Town
0,Aachtal,"Schrofenstrasse 19, 8580 Amriswil",8580,Amriswil
1,Aadorf,"Bahnhofstrasse 2, 8355 Aadorf",8355,Aadorf
2,Aarau,"Bahnhofplatz 4, 5000 Aarau",5000,Aarau
3,Aarau - Igelweid,"Igelweid 18, 5000 Aarau",5000,Aarau
4,Aarau Bahnhof,"Bahnhofplatz 3h, 5000 Aarau",5000,Aarau
5,Aathal Outlet,"Zürichstrasse 15, 8607 Aathal-Seegräben",8607,Aathal-Seegräben
6,Abtwil - Säntispark,"Wiesenbachstrasse 7, 9030 Abtwil",9030,Abtwil
7,Adligenswil LU,"Obgardirain 1, 6043 Adligenswil",6043,Adligenswil
8,Adliswil,"Albisstrasse 10, 8134 Adliswil",8134,Adliswil
9,Aesch,"Hauptstrasse 95, 4147 Aesch",4147,Aesch



✅ Total unique stores scraped: 1100


In [22]:
# df_stores

from datetime import datetime
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
df_stores.to_csv(f"migros_stores_{timestamp}.csv", index=False)

In [23]:
# await page.screenshot(path="screenshot.png")

## Denner